In [1]:
#!/usr/bin/env python

'''
use pymetadata module to create a minimal valid combine archive
using LEMS_NML2_Ex9_FN.sbml and LEMS_NML2_Ex9_FN.sedml
'''

import sys
sys.path.append("..")
import utils
import os
import pandas as pd
from IPython.display import display_markdown

sbml_filepath = 'LEMS_NML2_Ex9_FN.sbml'
sedml_filepath = 'LEMS_NML2_Ex9_FN_missing_xmlns.sedml' #xmlns:sbml missing

# For OpenCOR:https://docs.biosimulators.org/Biosimulators_OpenCOR/installation.html#python-package-and-command-line-program
# For rbapy: https://docs.biosimulators.org/Biosimulators_RBApy/installation.html#command-line-program
# smoldyn: https://smoldyn.readthedocs.io/en/latest/python/api.html#sed-ml-combine-biosimulators-api
# vcell https://github.com/virtualcell/vcell

# get folder that the current script is in

cwd = os.getcwd()
print(cwd)

c:\Users\prins\GitHub\SBMLShowcase\SBML


In [2]:
engines = utils.engines
types_dict = utils.types_dict

In [12]:
def check_file_compatibility(engine, types_dict, sbml_filepath, sedml_filepath):
    input_filetypes = set(utils.get_filetypes(sbml_filepath, sedml_filepath))
    input_file_types_text = [types_dict[i] for i in input_filetypes]

    engine_filetypes = engines[engine]
    if engine_filetypes is not None:
        # Flatten the list if the engine_filetypes is a list of tuples
        if all(isinstance(i, tuple) for i in engine_filetypes):
            engine_filetypes = {item for sublist in engine_filetypes for item in sublist}
        engine_file_types_text = [types_dict[i] for i in engine_filetypes if i in types_dict]
        if input_filetypes.issubset(engine_filetypes):
            return 'pass', (f"The file extensions suggest your file types '{input_file_types_text}' are compatible with {e}")
        else:
            return 'FAIL', (f"The file extensions suggest your file types '{input_file_types_text}' are not compatible with {e}. Try again using file types: {engine_file_types_text}")
    else:
        return 'FAIL', (f"{e} does not have any compatible file types.")



In [13]:
engine_dict = {}

for e in engines.keys():
    print('Running ' + e)
    output_dir = os.path.abspath(os.path.join('output', e))
    try:
        record = utils.run_biosimulators_docker(e, sedml_filepath, sbml_filepath, output_dir=output_dir)
        engine_dict[e] = record
    except Exception as error:
        error_message = str(error)
        print(f"Error occurred while running {e}")
        engine_dict[e] = error_message
        continue

utils.delete_output_folder('output')

Running amici
Running brian2
Running bionetgen
Running boolnet
Running cbmpy
Running cobrapy
Running copasi
Running gillespy2
Running ginsim
Running libsbmlsim
Running masspy
Running netpyne
Running neuron
Running opencor
Running pyneuroml
Running pysces
Running rbapy
Running smoldyn
Running tellurium
Running vcell
Running xpp


In [14]:
import re

def parse_error_message(text):
    if text != None:
        text_message = re.findall(r'"([^"]*)"', text) 
        if len(text_message) > 0:
            text = text_message
        else:
            text = text.replace('|', '')
            return text
        text = bytes(text[0], "utf-8").decode("unicode_escape")
        text = text.replace('|', '')

        # # for any text with "<*>" remove "<" as well as ">" but leave wildcard text *
        text = re.sub(r'<([^>]*)>', r'\1', text)

        # replace color codes with html color codes
        text = text.replace("\x1b[33m",'<span style="color:darkorange;">')
        text = text.replace("\x1b[31m",'<span style="color:red;">')

        # # remove .\x1b[0m
        text = text.replace("\x1b[0m", "")

        # find first "." or ":" after "<span*" and add "</span>"after it
        #text = re.sub(r'(<span style=*>?)[.:]?', r'\1. </span>', text)
        pattern = r'(<span style="[^"]*">[^.:]*)([.:])'
        replacement = r'\1\2</span>'
        text = re.sub(pattern, replacement, text, count=1)
        
        # text = re.sub(r'(<span style=*>?)[.:]?', r'\1. </span>', text)

        # bullet points and new lines
        text = text.replace('\r\n  - ', '</li><li>')
        text = text.replace('\r\n', '<br>')
        text = text.replace('\n', '<br>') 

        # BioSimulatorsWarning:  two <br> tags after
        text = text.replace('BioSimulatorsWarning:', '<br><br>BioSimulatorsWarning:<br><br>')
        text = text.replace('warnings.warn(termcolor.colored(message, Colors.warning.value), category)', '<br>')

        # if text includes The COMBINE/OMEX did not execute successfully: make everyhting from that point red
        text = text.replace('The COMBINE/OMEX did not execute successfully:', '<span style="color:red;">The COMBINE/OMEX did not execute successfully:')
    return text

def display_error_message(error_message):
    if error_message != None:
        display_markdown(f'{error_message}', raw=True)
    return error_message



In [15]:
# Create a table of the results
results_table = pd.DataFrame.from_dict(engine_dict).T
results_table.columns = ['pass/FAIL', 'Error']
results_table.index.name = 'Engine'
results_table.reset_index(inplace=True)

results_table['Error'] = results_table.apply(lambda x: None if x['pass/FAIL'] == x['Error'] else x['Error'], axis=1)
results_table['pass/FAIL'] = results_table['pass/FAIL'].replace('other', 'FAIL')
# # results_table['pass/FAIL'] = results_table['pass/FAIL'].apply(lambda x: x if x == 'FAIL' else x)

# for non empty cells in error col print error message

results_table['Error'] = results_table['Error'].apply(lambda x: utils.parse_error_message(x))
results_table['Error'] = results_table['Error'].apply(lambda x: utils.collapsible_content(x))

# # # use compatibility_message
results_table['Compatibility'] = results_table['Engine'].apply(lambda x: utils.check_file_compatibility_test(x, types_dict, sbml_filepath, sedml_filepath))
results_table['Compatibility'] = results_table['Compatibility'].apply(lambda x: utils.collapsible_content(x[1], title=x[0]))
results_table['pass/FAIL'] = results_table['pass/FAIL'].apply(lambda x: f'<span style="color:darkred;">{x}</span>' if x == 'FAIL' else x)
results_table['Compatibility'] = results_table['Compatibility'].apply(lambda x: f'<span style="color:darkred;">{x}</span>' if 'FAIL' in x else x)


results_md_table = results_table.to_markdown(index=False)
display_markdown(results_md_table, raw=True)

| Engine     | pass/FAIL                                | Error                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | Compatibility                                                                                                                                                                                                                         |
|:-----------|:-----------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| amici      | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| brian2     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| bionetgen  | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp680840` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in BNGL format (e.g., `sed:model/@language` must match `^urn:sedml:language:bngl(\.$)` such as `urn:sedml:language:bngl`).<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['BNGL', 'SED-ML']</details></span>            |
| boolnet    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp722711` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - Number of points (20000) must be equal to the difference between the output end (200.0) and start times (0.0).<br>R[write to console]: Warning message:<br><br>R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :<br>R[write to console]: <br> <br>R[write to console]:  library â/usr/lib/R/site-libraryâ contains no packages<br><br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['SBML-qual', 'SED-ML']</details></span>       |
| cbmpy      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><br>INFO: No xlwt module available, Excel spreadsheet creation disabled<br><br><br><br>No module named 'cplex'<br><br><br><br>CPLEX not available<br><br>*****<br>Using GLPK<br>*****<br><br><br>INFO: No xlrd module available, Excel spreadsheet reading disabled<br><br>CBMPy environment<br>******************<br>Revision: r689<br><br><br>***********************************************************************<br>* Welcome to CBMPy (0.7.25) - PySCeS Constraint Based Modelling       *<br>*                http://cbmpy.sourceforge.net                         *<br>* Copyright(C) Brett G. Olivier 2014 - 2019                           *<br>* Dept. of Systems Bioinformatics                                     *<br>* Vrije Universiteit Amsterdam, Amsterdam, The Netherlands            *<br>* CBMPy is developed as part of the BeBasic MetaToolKit Project       *<br>* Distributed under the GNU GPL v 3.0 licence, see                    *<br>* LICENCE (supplied with this release) for details                    *<br>***********************************************************************<br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp389318` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| cobrapy    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp894959` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| copasi     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp631862` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    could not convert string to float: 'Values[V]'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| gillespy2  | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| ginsim     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp872340` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - The interval between the output start and time time must be an integer multiple of the number of steps, not `0.01`:<br>          Output start time: 0.0<br>          Output end time: 200.0<br>          Number of steps: 20000<br></details> | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['SBML-qual', 'SED-ML']</details></span>       |
| libsbmlsim | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| masspy     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31 [1;31m<br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31 [1;31m<br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp986183` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `mass.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .<br>    	`(model, errors) = validate_sbml_model(filename)`<br>    If the model is valid and cannot be read please open an issue at https://github.com/SBRG/masspy/issues .<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| netpyne    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| neuron     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['SED-ML', 'LEMS', 'NeuroML']</details></span> |
| opencor    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| pyneuroml  | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['SED-ML', 'LEMS', 'NeuroML']</details></span> |
| pysces     | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>Could not find GLIMDA.<br>/usr/local/lib/python3.10/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` has warnings.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>      - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>      - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br>/usr/local/lib/python3.10/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive has warnings.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp354611` has warnings.<br>    - Model `net1` has warnings.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` has warnings.<br>        - 2 warnings of type SBML unit consistency (99505). The following is the first warning at line 40, column 12:<br>          - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>             The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        - 4 warnings of type Modeling practice (80701). The following is the first warning at line 32, column 12:<br>          - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>             The parameter with the id 'I' does not have a 'units' attribute.<br>  <br><br><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Model at /tmp/tmp_8w7iuwl/./LEMS_NML2_Ex9_FN.sbml could not be imported:<br>      <br>      File /tmp/tmp_8w7iuwl/./LEMS_NML2_Ex9_FN.sbml.xml does not exist<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| rbapy      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The SED document is potentially incorrect.</span></li><li>Model `net1` may be invalid.<br>    - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>         The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'I' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'SEC' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'V' does not have a 'units' attribute.<br>        <br>      - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>         The parameter with the id 'W' does not have a 'units' attribute.<br>        <br>  <br><br>/usr/local/lib/python3.9/site-packages/biosimulators_utils/warnings.py:31: <br><br>BioSimulatorsWarning:<br><br> <span style="color:darkorange;">The COMBINE/OMEX archive may be invalid.</li><li>OMEX manifests should not contain content entries for themselves.</li><li>The SED-ML file at location `./tmp418915` may be invalid.<br>    - Model `net1` may be invalid.<br>      - The model file `LEMS_NML2_Ex9_FN.sbml` may be invalid.<br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '(V - pow(V, 3) / 3 - W + I) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - In situations where a mathematical expression contains literal numbers or parameters whose units have not been declared, it is not possible to verify accurately the consistency of the units in the expression. <br>           The units of the rateRule math expression '0.08 * (V + (0.7 - 0.8 * W)) / SEC' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'I' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'SEC' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'V' does not have a 'units' attribute.<br>          <br>        - As a principle of best modeling practice, the units of a parameter should be declared rather than be left undefined. Doing so improves the ability of software to check the consistency of units and helps make it easier to detect potential errors in models.<br>           The parameter with the id 'W' does not have a 'units' attribute.<br>          <br>  <br><br><span style="color:red;"><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in RBA format (e.g., `sed:model/@language` must match `^urn:sedml:language:rba(\.$)` such as `urn:sedml:language:rba`).<br></details>                 | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['RBApy', 'SED-ML']</details></span>           |
| smoldyn    | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>[Errno 2] No such file or directory: '/tmp/tmpy86gjli0/model.txt'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | <span style="color:darkred;"><details><summary>FAIL</summary>xpp does not have any compatible file types.</details></span>                                                                                                            |
| tellurium  | pass                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | <details><summary>pass</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are compatible with xpp</details>                                                                                                    |
| vcell      | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/vcell' returned non-zero exit status 1: b'  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  PROCESSING_SEDML(preProcessDoc)  **** Error: SED-ML processing for /tmp/vcell_temp_4741e294-b1f9-40af-ae61-23dbe0e01d027251017393690650434/tmp948584 failed with error: \n  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  SIMULATIONS_RUN(runSimulations)  **** Error: \n  Root(root)  OMEX_EXECUTE(LEMS_NML2_Ex9_FN_missing_xmlns.omex)  PROCESSING_SIMULATION_OUTPUTS(processOutputs)  **** Error: Error processing outputs\n'```</details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | <span style="color:darkred;"><details><summary>FAIL</summary>xpp does not have any compatible file types.</details></span>                                                                                                            |
| xpp        | <span style="color:darkred;">FAIL</span> | <details><summary>Details</summary><span style="color:red;">No module named 'libsbml'<br></details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | <span style="color:darkred;"><details><summary>FAIL</summary>The file extensions suggest your file types '['SBML', 'SED-ML']' are not compatible with xpp. Try again using file types: ['XPP', 'SED-ML']</details></span>             |

In [16]:
# save results_md_table
with open('results.md', 'w', encoding='utf-8') as f:
    f.write(results_md_table)